In [1]:
import pickle
!pip install redis
import redis
redis_ip = '34.212.59.71'

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
r = redis.StrictRedis(redis_ip)
LSA = pickle.loads(r.get('tbl_LSA'))
vocabulary_expression = pickle.loads(r.get('tbl_vocabulary_expression'))

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.sparse as sparse
from scipy.spatial.distance import cosine
from spacy.en import English
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
LSA.head(2)

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_794,component_795,component_796,component_797,component_798,component_799,component_800,title,category,content
0,0.089703,-0.062023,0.025137,0.037325,-0.013474,0.025683,-0.007717,0.017116,-0.005195,-0.015903,...,0.004071,-0.023051,-0.006449,0.003322,0.010939,0.003897,-0.011495,(1+ε)-approximate nearest neighbor search,machine learning,approximate near neighbor search special case ...
1,0.180721,-0.183607,0.046931,0.148432,-0.034707,0.116325,0.029397,-0.003574,-0.066231,0.285732,...,-0.002134,0.026923,0.004420,0.002639,0.007617,0.005754,0.007940,ADALINE,machine learning,adaline adaptive linear neuron later adaptive ...


In [5]:
#  x train
LSA_components = LSA.drop(['category','title','content'], axis = 1)

In [6]:
# y train
cat = LSA['category']
cat_dummies = pd.get_dummies(cat)

In [7]:
dtc = DecisionTreeClassifier()
knc = KNeighborsClassifier()

In [8]:
param_dtc = {
    'max_depth':[3,6,9,12],
    'min_samples_leaf': [2,5,10,15]   
}

param_knc = {
     'n_neighbors': [5,10,15], 
     'leaf_size': [20,30]
}

In [9]:
gs_dtc = GridSearchCV(dtc, param_grid=param_dtc,cv=5,n_jobs = -1)
gs_dtc.fit(LSA_components, cat_dummies)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 6, 9, 12], 'min_samples_leaf': [2, 5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [10]:
gs_dtc_res_df = pd.DataFrame(gs_dtc.cv_results_)
gs_dtc_res_df.sort('rank_test_score',ascending=True).head(5).T[2:6]

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


,13,9,14,10,8
mean_test_score,0.520565,0.519682,0.512621,0.506796,0.500441
mean_train_score,0.940468,0.92767,0.911518,0.907282,0.939629
param_max_depth,12,9,12,9,9
param_min_samples_leaf,5,5,10,10,2


In [ ]:
gs_knc = GridSearchCV(knc, param_grid=param_knc,cv=5,n_jobs = -1)
gs_knc.fit(LSA_components, cat_dummies)

In [ ]:
gs_knc_res_df = pd.DataFrame(gs_knc.cv_results_)
gs_knc_res_df.sort('rank_test_score',ascending=True).head(5).T[2:6]